In [120]:
import numpy as np
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
from netCDF4 import num2date
import datetime

from tqdm import tqdm

tqdm.pandas()
from shapely.geometry import Point

import matplotlib.pyplot as plt

import platform

from cartopy import crs as ccrs

if platform.system() == 'Darwin':
    plt.rcParams["font.family"] = 'Arial Unicode MS'
elif platform.system() == 'Windows':
    plt.rcParams["font.family"] = 'SimHei'
else:
    pass

## 加载nc文件和读取数据


In [2]:
nc_data = nc.Dataset("cru_ts4.05.1901.2020.tmp.dat.nc")

In [206]:
for temp_value in nc_data.variables.values():
    print("*" * 70)
    print(temp_value)

**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    long_name: longitude
    units: degrees_east
unlimited dimensions: 
current shape = (720,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    long_name: latitude
    units: degrees_north
unlimited dimensions: 
current shape = (360,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 time(time)
    long_name: time
    units: days since 1900-1-1
    calendar: gregorian
unlimited dimensions: time
current shape = (1440,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 tmp(time, la

<ipython-input-206-ab2ee6009efa>:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(temp_value)


## 提取变量


In [4]:
raw_lat_data = np.array(nc_data.variables['lat'])
raw_lon_data = np.array(nc_data.variables['lon'])
raw_time_data = np.array(nc_data.variables['time'])
raw_tmp_data = np.array(nc_data.variables['tmp'])


<ipython-input-4-c2593932da71>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_lat_data = np.array(nc_data.variables['lat'])
<ipython-input-4-c2593932da71>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_lon_data = np.array(nc_data.variables['lon'])
<ipython-input-4-c2593932da71>:3: DeprecationWarning: `np.bool` is a deprecated alias for 

In [5]:
raw_tmp_data.shape


(1440, 360, 720)

In [6]:
def cftime2datetime(cftime, units, format='%Y-%m-%d %H:%M:%S'):
    """
    将nc文件里面的时间格式 从cftime 转换到 datetime格式
    :param cftime:
    :param units:
    :param format:
    :return:
    """
    return datetime.datetime.strptime(num2date(times=cftime, units=units).strftime(format), format)


clean_time_data = [cftime2datetime(i, units='days since 1900-1-1') for i in raw_time_data]
clean_time_data[:4]

[datetime.datetime(1901, 1, 16, 0, 0),
 datetime.datetime(1901, 2, 15, 0, 0),
 datetime.datetime(1901, 3, 16, 0, 0),
 datetime.datetime(1901, 4, 16, 0, 0)]

## 计算全球每年的平均气温

In [7]:
avg_mean_tmp = np.ones(shape=raw_tmp_data.shape[0])

for index in tqdm(range(raw_tmp_data.shape[0])):
    temp_temp = raw_tmp_data[index, :, :]
    avg_mean_tmp[index] = temp_temp[temp_temp != 9.96921e+36].mean()

100%|██████████| 1440/1440 [00:00<00:00, 2171.13it/s]


In [8]:
avg_mean_tmp_df = pd.DataFrame({'date': clean_time_data,
                                'avg_mean_tmp': avg_mean_tmp})
avg_mean_tmp_df['year'] = avg_mean_tmp_df['date'].dt.year
avg_mean_tmp_df['month'] = avg_mean_tmp_df['date'].dt.month
avg_mean_tmp_df


,date,avg_mean_tmp,year,month
0,1901-01-16,-2.276233,1901,1
1,1901-02-15,-0.606501,1901,2
2,1901-03-16,3.036036,1901,3
3,1901-04-16,7.728032,1901,4
4,1901-05-16,12.336249,1901,5
...,...,...,...,...
1435,2020-08-16,17.980717,2020,8
1436,2020-09-16,15.052033,2020,9
1437,2020-10-16,10.223289,2020,10
1438,2020-11-16,5.359230,2020,11


In [9]:
year_tmp_df = avg_mean_tmp_df.groupby(['year']).agg(
    avg_tmp=('avg_mean_tmp', 'mean')
).reset_index()
year_tmp_df.head()

,year,avg_tmp
0,1901,8.029477
1,1902,7.642335
2,1903,7.813257
3,1904,7.759539
4,1905,7.875768


In [11]:
% matplotlib

Using matplotlib backend: MacOSX


In [12]:
fig, ax = plt.subplots(figsize=(12, 4), dpi=200)
ax.plot(year_tmp_df['year'], year_tmp_df['avg_tmp'], linestyle='-', marker='o')
ax.set_title("全球各年平均气温,公众号：pypi")
ax.set_xlabel("年份")
ax.set_ylabel("温度平均数$ ^\circ C $")
plt.tight_layout()
plt.show()
fig.savefig("全球各年平均气温.png")

In [15]:
china_boundary = gpd.read_file(filename="中国地图边界202111版.json")

china_boundary_valid = china_boundary.copy()
china_boundary_valid['geometry'] = china_boundary.buffer(0)

In [16]:
fig, ax = plt.subplots()
china_boundary.boundary.plot(ax=ax)
ax.set_xlabel("longitude")
ax.set_ylabel("latitude")
ax.set_title("中国地图边界图")
plt.tight_layout()

### 对比一下两个地图

In [17]:
fig, ax = plt.subplots(ncols=2)
china_boundary.boundary.plot(ax=ax[0])
china_boundary_valid.boundary.plot(ax=ax[1])
ax[0].set_title("原始中国地图")
ax[1].set_title("处理有效后的中国地图")
plt.tight_layout()


In [85]:
def pic(lon, lat) -> bool:
    """
    检测一个点是否在中国边界线内
    lon:东经
    lat:北纬
    :param lon:
    :param lat:
    :return:
    """
    return china_boundary_valid.contains(Point(lon, lat))[0]


pic(lon=116, lat=45)

Lon_data, Lat_data = np.meshgrid(raw_lon_data, raw_lat_data)

point_set_df = pd.DataFrame({'longitude': Lon_data.flatten(),
                             'latitude': Lat_data.flatten(),
                             'tmp_300': raw_tmp_data[300, :, :].flatten()})

point_set_df.loc[point_set_df['tmp_300'] == np.max(raw_tmp_data), ['tmp_300']] = np.nan

In [98]:
clean_time_data[300]

datetime.datetime(1926, 1, 16, 0, 0)

In [86]:
# 检测每一个点是否在中国内

point_set_df['in_china'] = point_set_df.progress_apply(lambda x: pic(lon=x['longitude'], lat=x['latitude']), axis=1)
point_set_df.head()

100%|██████████| 259200/259200 [01:00<00:00, 4318.40it/s] 


,longitude,latitude,tmp_300,in_china
0,-179.75,-89.75,NaN,False
1,-179.25,-89.75,NaN,False
2,-178.75,-89.75,NaN,False
3,-178.25,-89.75,NaN,False
4,-177.75,-89.75,NaN,False


In [87]:
min(point_set_df['tmp_300']), max(point_set_df['tmp_300'])

(nan, nan)

## 可视化1(基础)

In [156]:
point_in_ch = point_set_df.loc[point_set_df['in_china']]

fig, ax = plt.subplots(figsize=(7, 5), dpi=200)
china_boundary.boundary.plot(ax=ax, color='black')
colorbar = ax.scatter(point_in_ch['longitude'], point_in_ch['latitude'],
                      c=point_in_ch['tmp_300'], s=3,
                      marker='s',
                      cmap=plt.cm.get_cmap('RdYlBu'))
ax.set_xlabel("longitude")
ax.set_ylabel("latitude")
ax.set_title(f"公众号：pypi 时间为：{clean_time_data[300]:%Y-%m}")
plt.tight_layout()
fig.colorbar(colorbar, ax=ax)

fig.savefig("检测点是否在中国境内.png")

## 可视化2(兰伯特投影)



In [174]:
china_center_x, china_center_y = china_boundary['geometry'].centroid.x, china_boundary['geometry'].centroid.y

<ipython-input-174-de275c171aaa>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  china_center_x, china_center_y = china_boundary['geometry'].centroid.x,china_boundary['geometry'].centroid.y
<ipython-input-174-de275c171aaa>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  china_center_x, china_center_y = china_boundary['geometry'].centroid.x,china_boundary['geometry'].centroid.y


103.87428899000118

In [175]:
crs_epsg = ccrs.LambertConformal(central_longitude=china_center_x[0], central_latitude=china_center_y[0])
china_boundary_ccrs = china_boundary.copy()
china_boundary_ccrs = china_boundary_ccrs.to_crs(crs_epsg.proj4_init)
# china_boundary_ccrs['geometry'].set_crs({'init': 'epsg:3875'},allow_override=True)
china_boundary_ccrs.crs


<Projected CRS: +proj=lcc +ellps=WGS84 +lon_0=103.8742889900012 +l ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Conic Conformal (2SP)
Datum: Unknown based on WGS84 ellipsoid
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [212]:
point_in_ch = point_set_df.loc[point_set_df['in_china']]

In [191]:
china_boundary.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [189]:
crs_epsg.proj4_init

'+ellps=WGS84 +proj=lcc +lon_0=103.8742889900012 +lat_0=36.42375089735982 +x_0=0.0 +y_0=0.0 +lat_1=33 +lat_2=45 +no_defs'

In [213]:
gpd_point_in_ch = gpd.GeoDataFrame(
    point_in_ch,
    geometry=gpd.points_from_xy(x=point_in_ch['longitude'], y=point_in_ch['latitude']),
    crs=china_boundary.crs
)
gpd_point_in_ch = gpd_point_in_ch.to_crs(crs_epsg.proj4_init)
gpd_point_in_ch

,longitude,latitude,tmp_300,in_china,geometry
138816,108.25,6.25,NaN,True,POINT (559727.940 -3501754.583)
140988,114.25,7.75,NaN,True,POINT (1303151.820 -3250214.180)
144588,114.25,10.25,NaN,True,POINT (1267582.939 -2940044.387)
153223,111.75,16.25,NaN,True,POINT (900246.991 -2247898.931)
156099,109.75,18.25,21.000000,True,POINT (656788.731 -2030169.810)
...,...,...,...,...,...
206524,122.25,53.25,-30.300001,True,POINT (1252555.828 2003950.741)
206525,122.75,53.25,-30.300001,True,POINT (1286152.735 2010934.895)
206526,123.25,53.25,-30.100000,True,POINT (1319710.706 2018103.797)
206527,123.75,53.25,-29.300001,True,POINT (1353228.725 2025457.229)


In [214]:
fig, ax = plt.subplots(figsize=(8, 7), dpi=150, subplot_kw={'projection': crs_epsg})
ax.gridlines(draw_labels=True,
             linewidth=2, color='gray', alpha=0.5, linestyle='--')
china_boundary_ccrs.boundary.plot(ax=ax, color='black', marker='s')
gpd_point_in_ch.plot(ax=ax, column='tmp_300', cmap=plt.cm.get_cmap('RdYlBu'), legend=True)
ax.set_xlabel("longitude")
ax.set_ylabel("latitude")
# ax.set_title(f"公众号：pypi 时间为：{clean_time_data[300]:%Y-%m}")
plt.tight_layout()

fig.savefig("检测点是否在中国境内_Lambert.png")

/opt/anaconda3/envs/devc/lib/python3.8/site-packages/cartopy/mpl/gridliner.py:531: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  elif len(intersection) > 4:
/opt/anaconda3/envs/devc/lib/python3.8/site-packages/cartopy/mpl/gridliner.py:543: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for inter in intersection:
IllegalArgumentException: Argument must be Polygonal or LinearRing
IllegalArgumentException: Argument must be Polygonal or LinearRing
IllegalArgumentException: Argument must be Polygonal or LinearRing
IllegalArgumentException: Argument must be Polygonal or LinearRing
IllegalArgumentException: Argument must be Polygonal or LinearRing
IllegalArgumentException: Arg

## 中国地图边界裁剪

In [20]:
china_boundary_box = china_boundary_valid.bounds.iloc[0]

In [208]:
fig, ax = plt.subplots(figsize=(7, 7), dpi=300)
china_boundary.boundary.plot(ax=ax, color='black')
# 在中国内的点
point_in_ch = point_set_df.loc[point_set_df['in_china']]
ax.scatter(point_in_ch['longitude'], point_in_ch['latitude'], c='red', s=1)

# 不在中国内的点
point_notin_ch = point_set_df.loc[~point_set_df['in_china']]
ax.scatter(point_notin_ch['longitude'], point_notin_ch['latitude'], c='blue', s=1)

ax.set_xlabel("longitude")
ax.set_ylabel("latitude")
ax.set_title("检测点是否在中国境内， 公众号：pypi")
ax.set_xlim(china_boundary_box.minx - 0.1, china_boundary_box.maxx + 0.1)
ax.set_ylim(china_boundary_box.miny - 0.1, china_boundary_box.maxy + 0.1)
plt.tight_layout()
fig.savefig("检测点是否在中国境内2.png")